# Python model for quick validation of Hardware Design Results and Software Results

对正向和反向各阶段的**输入**、**输出**以及重要的**参数**和**中间结果**做可视化打印，省去复杂的计算流程，主要包含：
- 正向
    - Tabulate       (table参数 R矩阵 S向量)
    - Descr          (R矩阵 G矩阵 scatter1/2 D矩阵)
    - FittingNet     (D矩阵 W[0~3]权重 B[0~3]偏置 idt参数)
- 反向
    - Descr_Grad     (D_Grad矩阵 scatter1/2^T导数)
    - Tabulate_Grad  (table参数 R_Grad S_Grad)

In [131]:
import numpy as np

## 1 正向 Tabulate 和 Descr
***
### 1.1 数据导入

In [132]:
def read_binary_tb(filename):
    table_sz = 1360 * 128 * 6
    s_sz = 108 * 512
    r_sz = 108 * 512 * 4
    fusa_sz = 108 * 4 * 128
    fusb_sz = 108 * 4 * 16
    descriptor_sz = 108 * 2048
    total_sz = table_sz + s_sz + r_sz + fusa_sz + fusb_sz + descriptor_sz
    
    data = np.fromfile(filename, dtype=np.float64, count=total_sz)
    if data.size != total_sz:
        print("binary file error\n")
    
    table = data[:table_sz].reshape((1360, 128, 6)).astype(np.float32)
    offset = table_sz
    
    s = data[offset:offset + s_sz].reshape((108, 512)).astype(np.float32)
    offset += s_sz
    
    r = data[offset:offset + r_sz].reshape((108, 512, 4)).astype(np.float32)
    offset += r_sz
    
    fusa = data[offset:offset + fusa_sz].reshape((108, 4, 128)).astype(np.float32)
    offset += fusa_sz
    
    fusb = data[offset:offset + fusb_sz].reshape((108, 4, 16)).astype(np.float32)
    offset += fusb_sz
    
    descriptor = data[offset:offset + descriptor_sz].reshape((108, 2048)).astype(np.float32)
    
    print("Table shape:", table.shape)
    print("S shape:", s.shape)
    print("R shape:", r.shape)
    print("Fusa shape:", fusa.shape)
    print("Fusb shape:", fusb.shape)
    print("Descriptor shape:", descriptor.shape)
    
    return table, s, r, fusa, fusb, descriptor

In [133]:
filename = "params/tb.txt"
table, s, r, fusa, fusb, descriptor = read_binary_tb(filename)
table = np.array(table)
s = np.array(s)
r = np.array(r)
fusa = np.array(fusa)
fusb = np.array(fusb)
descriptor = np.array(descriptor)

Table shape: (1360, 128, 6)
S shape: (108, 512)
R shape: (108, 512, 4)
Fusa shape: (108, 4, 128)
Fusb shape: (108, 4, 16)
Descriptor shape: (108, 2048)


***
### 1.2 打印显示 tabulate 和 descr 模块所需的计算参数

In [134]:
# 设置打印精度
# 修改长度即可
def custom_format(x):
    return "{:.6e}".format(x)

np.set_printoptions(formatter={'float_kind': custom_format})

In [135]:
# print(table)
index = 0
print(table)

[[[4.784702e-02 3.074703e-02 9.051514e-04 -1.762748e-03 -1.926316e-03
   2.166739e-04]
  [2.760558e+00 -4.357794e-01 -3.426212e-01 -1.141476e-01 3.229325e-02
   5.569158e-02]
  [-1.289653e+00 1.970249e-01 -5.566421e-02 5.305465e-03 1.410724e-03
   -5.534474e-04]
  ...
  [1.023927e-03 -5.640802e-03 2.640829e-05 3.406440e-05 -3.207621e-06
   -2.734040e-05]
  [9.850942e-01 -1.379079e-02 -6.145815e-03 -1.848618e-03 -4.095307e-04
   -5.129868e-05]
  [-1.593706e-02 2.138825e-03 2.060935e-03 3.662213e-03 1.195831e-03
   -3.138275e-03]]

 [[4.815458e-02 3.076460e-02 8.511154e-04 -1.839582e-03 -1.915395e-03
   3.185766e-04]
  [2.756166e+00 -4.426659e-01 -3.460257e-01 -1.128001e-01 3.507810e-02
   5.703956e-02]
  [-1.287688e+00 1.959132e-01 -5.550421e-02 5.361337e-03 1.382992e-03
   -5.735286e-04]
  ...
  [9.675220e-04 -5.640263e-03 2.742802e-05 3.390903e-05 -4.392139e-06
   -2.736973e-05]
  [9.849557e-01 -1.391426e-02 -6.201520e-03 -1.865048e-03 -4.120806e-04
   -3.682566e-05]
  [-1.591546e-02 

In [136]:
# print(s)
index = 0
print(s[0].shape)
print(s.dtype)
print(s)

(512,)
float32
[[-1.366627e-01 -1.387978e-01 -1.452103e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]
 [-1.238860e-01 -1.279753e-01 -1.343581e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]
 [1.627834e+00 -1.303367e-01 -1.303367e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]
 ...
 [-1.243136e-01 -1.285512e-01 -1.296193e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]
 [-1.254131e-01 -1.348833e-01 -1.398551e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]
 [-1.238175e-01 -1.243104e-01 -1.245672e-01 ... -3.114757e-01
  -3.114757e-01 -3.114757e-01]]


In [137]:
# print(r)
index = 0
print(r[index].shape, r[index].dtype)
print(r)

(512, 4) float32
[[[-1.366627e-01 -7.842806e-02 -1.549703e-01 -2.310936e-01]
  [-1.387978e-01 -2.234224e-01 1.593752e-01 7.889253e-02]
  [-1.452103e-01 2.186190e-01 -6.887148e-02 -1.518576e-01]
  ...
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]]

 [[-1.238860e-01 8.223561e-02 1.644414e-01 2.498594e-01]
  [-1.279753e-01 -8.247689e-02 -2.422139e-01 1.631367e-01]
  [-1.343581e-01 -1.543723e-01 -2.359258e-01 -7.935484e-02]
  ...
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]]

 [[1.627834e+00 1.388824e+00 -1.092173e+00 2.676429e+00]
  [-1.303367e-01 -1.579775e-01 -2.386146e-01 8.851583e-02]
  [-1.303367e-01 -1.622300e-01 -7.448687e-02 -2.405450e-01]
  ...
  [-3.114757e-01 -0.000000e+00 -0.000000e+00 -0.000000e+00]
  [-3.114757

In [138]:
# print(fusa)
index = 0
print(fusa[0].shape)
print(fusa)

(4, 128)
[[[1.074825e-02 -2.314112e-01 -2.471243e-02 ... -7.135562e-03
   -1.735821e-01 2.604660e-02]
  [-6.214165e-04 -4.634155e-05 -2.166054e-04 ... -1.621325e-05
   9.138225e-04 5.636419e-04]
  [2.762329e-03 -3.790419e-03 3.317517e-03 ... 1.724540e-04 2.300115e-03
   -9.119314e-04]
  [-7.604534e-05 -2.982423e-04 7.931971e-04 ... 3.320595e-06
   -1.217879e-03 -1.934433e-03]]

 [[1.212052e-02 -2.308901e-01 -2.482706e-02 ... -7.104613e-03
   -1.718562e-01 2.815243e-02]
  [-1.467297e-03 -2.199057e-03 2.054156e-03 ... 3.485668e-05
   -5.612059e-04 2.342322e-03]
  [3.676020e-04 2.374203e-04 -3.326370e-04 ... 6.514284e-06 8.548853e-04
   -2.250953e-04]
  [-1.412478e-03 -4.858969e-04 5.738969e-04 ... -4.792697e-05
   -2.380579e-03 1.188796e-03]]

 [[7.890423e-03 -2.308443e-01 -2.505542e-02 ... -7.202014e-03
   -1.738555e-01 2.627745e-02]
  [3.230689e-03 -4.789916e-03 4.832520e-03 ... 2.014177e-04
   -4.441689e-04 1.537342e-03]
  [3.636491e-03 -4.089372e-03 3.600906e-03 ... 1.776291e-04 3.36

In [139]:
# print(fusb)
index = 0
print(fusb[0].shape)
print(fusb)

(4, 16)
[[[1.074825e-02 -2.314112e-01 -2.471243e-02 ... -2.302068e-03
   -3.034864e-03 3.332170e-05]
  [-6.214165e-04 -4.634155e-05 -2.166054e-04 ... -1.435351e-06
   -4.809337e-05 2.869256e-06]
  [2.762329e-03 -3.790419e-03 3.317517e-03 ... 1.783962e-05 1.809837e-03
   3.479242e-04]
  [-7.604534e-05 -2.982423e-04 7.931971e-04 ... -1.058759e-05
   -3.573858e-05 -7.717720e-05]]

 [[1.212052e-02 -2.308901e-01 -2.482706e-02 ... -2.291130e-03
   -2.085069e-03 1.822600e-04]
  [-1.467297e-03 -2.199057e-03 2.054156e-03 ... -1.119427e-05
   -5.127686e-04 -5.056987e-05]
  [3.676020e-04 2.374203e-04 -3.326370e-04 ... 6.979323e-06 1.705580e-04
   6.187447e-05]
  [-1.412478e-03 -4.858969e-04 5.738969e-04 ... -2.481663e-05
   -8.685274e-04 -2.238605e-04]]

 [[7.890423e-03 -2.308443e-01 -2.505542e-02 ... -2.317427e-03
   -3.763606e-03 -9.699066e-05]
  [3.230689e-03 -4.789916e-03 4.832520e-03 ... 5.462398e-06 1.487733e-03
   2.289228e-04]
  [3.636491e-03 -4.089372e-03 3.600906e-03 ... 1.288795e-05 1.

In [140]:
# 通过fusa和fusb计算d
d = np.zeros((108, 128, 16))
for i in range(108):
    d[i] = np.dot(fusa[i].T, fusb[i])
d = d.reshape((108, 2048))
print(d.shape)
print(d[0, 0:20])

(108, 2048)
[1.235473e-04 -2.497684e-03 -2.563770e-04 1.239485e-04 4.109494e-04
 -5.660573e-04 -2.766176e-04 2.532252e-04 4.719249e-03 -1.203750e-03
 -5.247078e-05 1.040506e-03 -1.759414e-06 -2.469222e-05 -2.758750e-05
 1.323317e-06 -2.497684e-03 5.356561e-02 5.705933e-03 -2.598294e-03]


In [141]:
# 软件代码算完的d
index = 0
print(descriptor.shape, descriptor.dtype)
print(descriptor[index, 0:20])

(108, 2048) float32
[1.235473e-04 -2.497684e-03 -2.563770e-04 1.239485e-04 4.109494e-04
 -5.660573e-04 -2.766176e-04 2.532252e-04 4.719249e-03 -1.203750e-03
 -5.247079e-05 1.040506e-03 -1.759414e-06 -2.469222e-05 -2.758750e-05
 1.323317e-06 -2.497684e-03 5.356561e-02 5.705933e-03 -2.598294e-03]


***
## 2 打印 fitting net 参数以及中间结果

### 2.1 fitting net 参数导入

In [142]:
def read_binary_fn(filename):
    # 定义每个部分的大小
    w0_sz = 2048 * 240
    b0_sz = 240
    w1_sz = 240 * 240
    b1_sz = 240
    w2_sz = 240 * 240
    b2_sz = 240
    w3_sz = 240
    b3_sz = 1
    idt1_sz = 240
    idt2_sz = 240
    total_sz = w0_sz + b0_sz + w1_sz + b1_sz + w2_sz + b2_sz + w3_sz + b3_sz + idt1_sz + idt2_sz
    
    # 读取二进制文件中的数据
    data = np.fromfile(filename, dtype=np.float64, count=total_sz)
    
    # 验证读取的数据总量是否正确
    if data.size != total_sz:
        raise ValueError(f"Read data size {data.size} does not match expected size {total_sz}")
    
    # 按照定义的大小截取数据并重塑
    offset = 0
    
    w0 = data[offset:offset + w0_sz].reshape((2048, 240)).astype(np.float32)
    offset += w0_sz
    
    b0 = data[offset:offset + b0_sz].astype(np.float32)
    offset += b0_sz
    
    w1 = data[offset:offset + w1_sz].reshape((240, 240)).astype(np.float32)
    offset += w1_sz
    
    b1 = data[offset:offset + b1_sz].astype(np.float32)
    offset += b1_sz
    
    w2 = data[offset:offset + w2_sz].reshape((240, 240)).astype(np.float32)
    offset += w2_sz
    
    b2 = data[offset:offset + b2_sz].astype(np.float32)
    offset += b2_sz
    
    w3 = data[offset:offset + w3_sz].reshape((240, 1)).astype(np.float32)
    offset += w3_sz
    
    b3 = data[offset:offset + b3_sz].astype(np.float32)
    offset += b3_sz
    
    idt1 = data[offset:offset + idt1_sz].astype(np.float32)
    offset += idt1_sz
    
    idt2 = data[offset:offset + idt2_sz].astype(np.float32)
    
    # 打印调试信息
    print("Shapes of the arrays:")
    print("w0 shape:", w0.shape)
    print("b0 shape:", b0.shape)
    print("w1 shape:", w1.shape)
    print("b1 shape:", b1.shape)
    print("w2 shape:", w2.shape)
    print("b2 shape:", b2.shape)
    print("w3 shape:", w3.shape)
    print("b3 shape:", b3.shape)
    print("idt1 shape:", idt1.shape)
    print("idt2 shape:", idt2.shape)
    
    return w0, b0, w1, b1, w2, b2, w3, b3, idt1, idt2

In [143]:
filename = 'params/fn.txt'
w0, b0, w1, b1, w2, b2, w3, b3, idt1, idt2 = read_binary_fn(filename)
w0 = np.array(w0)
b0 = np.array(b0)
w1 = np.array(w1)
b1 = np.array(b1)
w2 = np.array(w2)
b2 = np.array(b2)
w3 = np.array(w3)
b3 = np.array(b3)
idt1 = np.array(idt1)
idt2 = np.array(idt2)

Shapes of the arrays:
w0 shape: (2048, 240)
b0 shape: (240,)
w1 shape: (240, 240)
b1 shape: (240,)
w2 shape: (240, 240)
b2 shape: (240,)
w3 shape: (240, 1)
b3 shape: (1,)
idt1 shape: (240,)
idt2 shape: (240,)


***
### 2.2 输出结果浮点数到文件

In [144]:
# 输出到文件
def write_fp32_file(filename, array):
    with open(filename, 'w') as f:
        for num in array.flatten():
            f.write(f"{num}\n")

In [145]:
write_fp32_file("fp32/fn32fp/descriptor.txt", descriptor)
write_fp32_file("fp32/fn32fp/w0.txt", w0)
write_fp32_file("fp32/fn32fp/b0.txt", b0)
write_fp32_file("fp32/fn32fp/w1.txt", w1)
write_fp32_file("fp32/fn32fp/b1.txt", b1)
write_fp32_file("fp32/fn32fp/w2.txt", w2)
write_fp32_file("fp32/fn32fp/b2.txt", b2)
write_fp32_file("fp32/fn32fp/w3.txt", w3)
write_fp32_file("fp32/fn32fp/b3.txt", b3)
write_fp32_file("fp32/fn32fp/idt1.txt", idt1)
write_fp32_file("fp32/fn32fp/idt2.txt", idt2)

***
### 2.3 输出二进制形式/十六进制形式到文件
接口：
- write_bin_file(filename, array)
- write_hex_file(filename, array)
    - filename 保存写入的文件名
    - array    需要保存的数据数组

In [146]:
# 打印01文件
import struct
def float32_to_bin(num):
    packed = struct.pack('!f', num)
    integers = struct.unpack('!I', packed)[0]
    bin_str = format(integers, '032b')
    return bin_str

def write_bin_file(filename, array):
    with open(filename, 'w') as f:
        for num in array.flatten():
            bin_str = float32_to_bin(num)
            f.write(bin_str + '\n')
# 十六进制
def float32_to_hex(num):
    packed = struct.pack('!f', num)
    integers = struct.unpack('!I', packed)[0]
    bin_str = format(integers, '08X')
    return bin_str

def write_hex_file(filename, array):
    with open(filename, 'w') as f:
        for num in array.flatten():
            bin_str = float32_to_hex(num)
            f.write(bin_str + '\n')

In [147]:
# 输出二进制
write_bin_file("fp32/fn32bin/descriptor.txt", descriptor)
write_bin_file("fp32/fn32bin/w0.txt", w0)
write_bin_file("fp32/fn32bin/b0.txt", b0)
write_bin_file("fp32/fn32bin/w1.txt", w1)
write_bin_file("fp32/fn32bin/b1.txt", b1)
write_bin_file("fp32/fn32bin/w2.txt", w2)
write_bin_file("fp32/fn32bin/b2.txt", b2)
write_bin_file("fp32/fn32bin/w3.txt", w3)
write_bin_file("fp32/fn32bin/b3.txt", b3)
write_bin_file("fp32/fn32bin/idt1.txt", idt1)
write_bin_file("fp32/fn32bin/idt2.txt", idt2)

In [148]:
# 输出十六进制
write_hex_file("fp32/fn32hex/descriptor.txt", descriptor)
write_hex_file("fp32/fn32hex/w0.txt", w0)
write_hex_file("fp32/fn32hex/b0.txt", b0)
write_hex_file("fp32/fn32hex/w1.txt", w1)
write_hex_file("fp32/fn32hex/b1.txt", b1)
write_hex_file("fp32/fn32hex/w2.txt", w2)
write_hex_file("fp32/fn32hex/b2.txt", b2)
write_hex_file("fp32/fn32hex/w3.txt", w3)
write_hex_file("fp32/fn32hex/b3.txt", b3)
write_hex_file("fp32/fn32hex/idt1.txt", idt1)
write_hex_file("fp32/fn32hex/idt2.txt", idt2)

***
### 2.4 显示打印fn数据

In [149]:
# print(w0)
# print(b0)
# print(w1)
# print(b1)
# print(w2)
# print(b2)
# print(w3)
print(b3)
# print(idt1)
# print(idt2)

[-3.895333e+00]


In [150]:
print(idt1)

[5.008643e-01 -3.851940e-01 5.126717e-01 -1.713570e-01 3.136898e-01
 4.114259e-01 -4.658281e-01 -4.506604e-01 6.054414e-01 -3.633977e-01
 5.682402e-01 5.492365e-01 5.072284e-01 5.364794e-01 -4.985216e-01
 -4.267056e-01 3.972457e-01 5.110252e-01 -4.097421e-01 5.046139e-01
 -1.622452e-01 -5.226681e-01 4.889802e-01 5.963080e-01 4.660241e-01
 -4.165389e-01 -5.285802e-01 -1.063399e+00 -3.918626e-01 5.353228e-01
 4.710637e-01 -4.361430e-01 -5.257453e-02 -4.441959e-01 4.706535e-01
 6.588836e-01 -4.762637e-01 -3.638220e-01 -4.643600e-01 -4.859901e-01
 -4.712907e-01 6.129452e-01 5.213938e-01 5.390962e-01 5.702167e-01
 5.232896e-01 4.800342e-01 1.002972e+00 4.866808e-01 -5.481529e-01
 -7.592868e-01 -4.291524e-01 -4.927672e-01 -4.598127e-01 5.012403e-01
 -5.056809e-01 -4.746830e-01 -4.152530e-01 -5.057840e-01 -4.222319e-01
 5.058115e-01 5.816525e-01 -3.853152e-01 5.138348e-01 4.764221e-01
 -4.947608e-01 5.667351e-01 4.993246e-01 -3.937530e-01 3.258433e-01
 3.746129e-01 5.993138e-01 -5.313031e-01 

In [151]:
print(idt2)

[4.991365e-01 6.148065e-01 -4.873284e-01 2.327460e-01 -1.349843e-01
 5.885720e-01 -5.341744e-01 5.493373e-01 -3.945575e-01 6.366029e-01
 -4.317583e-01 -4.507615e-01 4.927697e-01 -4.635202e-01 -5.014794e-01
 5.732944e-01 -2.403676e-01 4.889734e-01 5.902562e-01 4.953852e-01
 -2.035065e-01 -4.773336e-01 5.110188e-01 4.036934e-01 5.339746e-01
 5.834598e-01 -4.714191e-01 6.340014e-02 6.081371e-01 -4.236625e-01
 -5.289354e-01 5.638567e-01 1.287999e-01 5.558041e-01 5.293471e-01
 -3.411153e-01 5.237371e-01 -6.361771e-01 -5.356395e-01 -5.140116e-01
 5.287076e-01 -3.870543e-01 4.786049e-01 4.609013e-01 -4.297863e-01
 -4.767157e-01 4.919688e-01 2.970705e-03 5.133195e-01 -4.518512e-01
 -2.407120e-01 5.708470e-01 -5.072313e-01 -5.401865e-01 4.987617e-01
 -4.943171e-01 5.253181e-01 -3.943024e-01 -4.942171e-01 5.777684e-01
 4.941885e-01 -4.183464e-01 6.146874e-01 4.861648e-01 5.235821e-01
 -5.052390e-01 -4.332637e-01 -5.006749e-01 6.062450e-01 6.741552e-01
 5.600216e-01 -4.006839e-01 -4.686983e-01 -5

### 2.5 fitting net 中间流程以及每一步结果
模拟fittingnet实际流程->构建每一步的python model数据验证
也可以使用上面的打印接口全部打印到文件

In [152]:
# layer 0
l0_mul = np.dot(descriptor, w0) + b0
print(l0_mul)

[[-5.908176e+00 -5.871770e+00 -6.224822e+00 ... -6.123595e+00
  6.273276e+00 -5.852995e+00]
 [-5.856635e+00 -5.839114e+00 -6.198472e+00 ... -6.088060e+00
  6.222655e+00 -5.816985e+00]
 [-5.903490e+00 -5.868115e+00 -6.223085e+00 ... -6.114127e+00
  6.289127e+00 -5.854053e+00]
 ...
 [-5.886058e+00 -5.848067e+00 -6.211116e+00 ... -6.092611e+00
  6.260082e+00 -5.833415e+00]
 [-5.891986e+00 -5.857602e+00 -6.203376e+00 ... -6.098969e+00
  6.267869e+00 -5.839854e+00]
 [-5.821606e+00 -5.809018e+00 -6.174389e+00 ... -6.051877e+00
  6.186563e+00 -5.784042e+00]]


In [153]:
l0_tanh = np.tanh(l0_mul)
print(l0_tanh.shape, l0_tanh)

(108, 240) [[-9.999853e-01 -9.999841e-01 -9.999921e-01 ... -9.999904e-01
  9.999928e-01 -9.999835e-01]
 [-9.999837e-01 -9.999831e-01 -9.999917e-01 ... -9.999897e-01
  9.999921e-01 -9.999823e-01]
 [-9.999851e-01 -9.999840e-01 -9.999921e-01 ... -9.999902e-01
  9.999931e-01 -9.999835e-01]
 ...
 [-9.999846e-01 -9.999834e-01 -9.999920e-01 ... -9.999898e-01
  9.999927e-01 -9.999829e-01]
 [-9.999848e-01 -9.999837e-01 -9.999918e-01 ... -9.999899e-01
  9.999928e-01 -9.999831e-01]
 [-9.999825e-01 -9.999820e-01 -9.999913e-01 ... -9.999889e-01
  9.999915e-01 -9.999811e-01]]


In [154]:
# layer1
l1_mul = np.dot(l0_tanh, w1) + b1
print(l1_mul)

[[1.054258e+01 -1.392344e+01 1.065050e+01 ... -1.064122e+01 1.112427e+01
  -1.111186e+01]
 [1.053369e+01 -1.392200e+01 1.065075e+01 ... -1.063416e+01 1.112132e+01
  -1.110108e+01]
 [1.055345e+01 -1.393575e+01 1.065828e+01 ... -1.064759e+01 1.112379e+01
  -1.112170e+01]
 ...
 [1.053998e+01 -1.392075e+01 1.065196e+01 ... -1.063755e+01 1.111654e+01
  -1.110214e+01]
 [1.054759e+01 -1.392612e+01 1.064951e+01 ... -1.063837e+01 1.112272e+01
  -1.111292e+01]
 [1.052457e+01 -1.392186e+01 1.065333e+01 ... -1.062638e+01 1.111674e+01
  -1.109242e+01]]


In [155]:
l1_tanh = np.tanh(l1_mul)
print(l1_tanh)

[[1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]
 [1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]
 [1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]
 ...
 [1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]
 [1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]
 [1.000000e+00 -1.000000e+00 1.000000e+00 ... -1.000000e+00 1.000000e+00
  -1.000000e+00]]


In [156]:
l1_idt = l1_tanh * idt1
print(l1_idt)

[[5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]
 [5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]
 [5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]
 ...
 [5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]
 [5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]
 [5.008643e-01 3.851940e-01 5.126717e-01 ... 4.614599e-01 -4.519329e-01
  3.840962e-01]]


In [157]:
l1_add = l1_idt + l0_tanh
print(l1_add)

[[-4.991210e-01 -6.147901e-01 -4.873204e-01 ... -5.385305e-01
  5.480599e-01 -6.158873e-01]
 [-4.991193e-01 -6.147890e-01 -4.873200e-01 ... -5.385298e-01
  5.480592e-01 -6.158861e-01]
 [-4.991208e-01 -6.147900e-01 -4.873204e-01 ... -5.385303e-01
  5.480602e-01 -6.158873e-01]
 ...
 [-4.991202e-01 -6.147894e-01 -4.873202e-01 ... -5.385299e-01
  5.480597e-01 -6.158867e-01]
 [-4.991205e-01 -6.147896e-01 -4.873201e-01 ... -5.385300e-01
  5.480598e-01 -6.158869e-01]
 [-4.991181e-01 -6.147879e-01 -4.873196e-01 ... -5.385290e-01
  5.480586e-01 -6.158849e-01]]


In [158]:
# layer2
l2_mul = np.dot(l1_add, w2) + b2
print(l2_mul[0])

[8.705782e+00 9.773945e+00 -9.680218e+00 1.121768e+01 9.972608e+00
 -9.605145e+00 -9.685271e+00 -9.087512e+00 -8.729086e+00 -1.076654e+01
 8.829419e+00 -9.167223e+00 -9.242976e+00 8.808874e+00 9.352021e+00
 -1.024829e+01 1.005426e+01 9.314691e+00 9.913836e+00 8.905868e+00
 1.088768e+01 9.589687e+00 -1.127421e+01 -1.028721e+01 8.808594e+00
 8.924168e+00 -1.028486e+01 7.890327e+00 -8.979429e+00 -1.049288e+01
 -9.090010e+00 9.757996e+00 1.070431e+01 -9.455331e+00 -9.178235e+00
 -9.907694e+00 1.051991e+01 -9.459115e+00 -9.271498e+00 -9.439185e+00
 -9.462749e+00 9.292680e+00 -9.124596e+00 1.026376e+01 -8.824150e+00
 9.234430e+00 1.128937e+01 -9.279321e+00 9.324448e+00 -9.764970e+00
 8.052547e+00 -8.844540e+00 1.029956e+01 1.113130e+01 -9.861071e+00
 8.748677e+00 -9.751061e+00 -1.068080e+01 9.611946e+00 -9.688925e+00
 9.995170e+00 -1.028796e+01 -8.780681e+00 -8.717286e+00 -8.914675e+00
 -9.374828e+00 -8.863949e+00 9.317116e+00 9.221182e+00 -9.315438e+00
 1.074939e+01 8.979362e+00 9.202559e+0

In [159]:
l2_tanh = np.tanh(l2_mul)
print(l2_tanh)

[[9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]
 [9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]
 [9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]
 ...
 [9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]
 [9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]
 [9.999999e-01 9.999999e-01 -9.999999e-01 ... 1.000000e+00 9.999999e-01
  9.999999e-01]]


In [160]:
l2_idt = l2_tanh * idt2
print(l2_idt)

[[4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]
 [4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]
 [4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]
 ...
 [4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]
 [4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]
 [4.991365e-01 6.148064e-01 4.873284e-01 ... 5.385409e-01 -5.480661e-01
  6.159046e-01]]


In [161]:
l2_add = l2_idt + l1_add
print(l2_add)

[[1.552701e-05 1.633167e-05 7.927418e-06 ... 1.043081e-05 -6.139278e-06
  1.728535e-05]
 [1.713634e-05 1.740456e-05 8.344650e-06 ... 1.114607e-05 -6.854534e-06
  1.853704e-05]
 [1.570582e-05 1.645088e-05 7.927418e-06 ... 1.055002e-05 -5.900860e-06
  1.728535e-05]
 ...
 [1.624227e-05 1.704693e-05 8.106232e-06 ... 1.102686e-05 -6.377697e-06
  1.794100e-05]
 [1.600385e-05 1.680851e-05 8.225441e-06 ... 1.090765e-05 -6.258488e-06
  1.776218e-05]
 [1.832843e-05 1.847744e-05 8.761883e-06 ... 1.186132e-05 -7.450581e-06
  1.972914e-05]]


In [162]:
# layer3
lf = np.dot(l2_add, w3) + b3
print(lf[0:10])

[[-3.712208e+00]
 [-3.702184e+00]
 [-3.638035e+00]
 [-3.677732e+00]
 [-3.657131e+00]
 [-3.720031e+00]
 [-3.689260e+00]
 [-3.680140e+00]
 [-3.698677e+00]
 [-3.690333e+00]]


In [163]:
energy = np.sum(lf)
print(energy)

-398.07773


In [164]:
#layer2 grad
lf_grad = np.ones((108, 1))
l2_grad_mul = np.dot(lf_grad, w3.T)
print(l2_grad_mul)

[[1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]
 [1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]
 [1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]
 ...
 [1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]
 [1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]
 [1.066306e-05 8.584118e-06 9.105376e-07 ... 4.095716e-06 -6.757952e-08
  1.632654e-05]]


In [165]:
l2_grad_idt = l2_grad_mul * idt2
print(l2_grad_idt)

[[5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]
 [5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]
 [5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]
 ...
 [5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]
 [5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]
 [5.322325e-06 5.277572e-06 -4.437308e-07 ... 2.205711e-06 3.703805e-08
  1.005559e-05]]


In [166]:
def tanh_grad(y, dy):
    return dy * (1.0 - y ** 2)

In [167]:
l2_grad_tanh = tanh_grad(l2_tanh, l2_grad_idt)
print(l2_grad_tanh)

[[6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]
 [6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]
 [6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]
 ...
 [6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]
 [6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]
 [6.344705e-13 6.291356e-13 -5.289684e-14 ... 0.000000e+00 4.415279e-15
  1.198720e-12]]


In [168]:
# layer1_grad
l1_grad_mul = np.dot(l2_grad_tanh, w2.T)
print(l1_grad_mul)

[[-6.996883e-10 -9.832730e-10 -2.123710e-09 ... -1.973881e-09
  1.072767e-09 1.329942e-10]
 [-6.996836e-10 -9.832657e-10 -2.123709e-09 ... -1.973870e-09
  1.072750e-09 1.330126e-10]
 [-6.996429e-10 -9.832160e-10 -2.123684e-09 ... -1.973781e-09
  1.072728e-09 1.329324e-10]
 ...
 [-6.996867e-10 -9.832753e-10 -2.123707e-09 ... -1.973879e-09
  1.072770e-09 1.329871e-10]
 [-6.996836e-10 -9.832657e-10 -2.123709e-09 ... -1.973870e-09
  1.072750e-09 1.330126e-10]
 [-6.996815e-10 -9.832592e-10 -2.123711e-09 ... -1.973865e-09
  1.072737e-09 1.330296e-10]]


In [169]:
l1_grad_add = l1_grad_mul + l2_grad_mul
print(l1_grad_add)

[[1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650675e-08
  1.632667e-05]
 [1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650677e-08
  1.632667e-05]
 [1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650679e-08
  1.632667e-05]
 ...
 [1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650675e-08
  1.632667e-05]
 [1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650677e-08
  1.632667e-05]
 [1.066236e-05 8.583135e-06 9.084139e-07 ... 4.093742e-06 -6.650678e-08
  1.632667e-05]]


In [170]:
l1_grad_idt = l1_grad_add * idt1
print(l1_grad_idt)

[[5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005659e-08
  -6.271012e-06]
 [5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005660e-08
  -6.271012e-06]
 [5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005661e-08
  -6.271012e-06]
 ...
 [5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005659e-08
  -6.271012e-06]
 [5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005660e-08
  -6.271012e-06]
 [5.340398e-06 -3.306172e-06 4.657181e-07 ... -1.889098e-06 3.005661e-08
  -6.271012e-06]]


In [171]:
l1_grad_tanh = tanh_grad(l1_tanh, l1_grad_idt)
print(l1_grad_tanh)

[[0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]
 [0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]
 [0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]
 ...
 [0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]
 [0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]
 [0.000000e+00 -0.000000e+00 0.000000e+00 ... -0.000000e+00 0.000000e+00
  -0.000000e+00]]


In [172]:
# layer0 grad
l0_grad_mul = np.dot(l1_grad_tanh, w1.T)
print(l0_grad_mul.shape)
print(l0_grad_mul)

(108, 240)
[[-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]
 [-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]
 [-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]
 ...
 [-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]
 [-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]
 [-3.772370e-10 2.869110e-10 4.491451e-10 ... 9.614258e-10 -9.651740e-10
  6.737897e-10]]


In [173]:
l0_grad_add = l0_grad_mul + l1_grad_add
print(l0_grad_add)

[[1.066199e-05 8.583422e-06 9.088630e-07 ... 4.094704e-06 -6.747193e-08
  1.632734e-05]
 [1.066199e-05 8.583422e-06 9.088630e-07 ... 4.094704e-06 -6.747194e-08
  1.632734e-05]
 [1.066199e-05 8.583422e-06 9.088631e-07 ... 4.094704e-06 -6.747197e-08
  1.632734e-05]
 ...
 [1.066199e-05 8.583422e-06 9.088630e-07 ... 4.094704e-06 -6.747192e-08
  1.632734e-05]
 [1.066199e-05 8.583422e-06 9.088630e-07 ... 4.094704e-06 -6.747194e-08
  1.632734e-05]
 [1.066199e-05 8.583422e-06 9.088630e-07 ... 4.094704e-06 -6.747196e-08
  1.632734e-05]]


In [174]:
l0_grad_tanh = tanh_grad(l0_tanh, l0_grad_add)
print(l0_grad_tanh.shape)
print(l0_grad_tanh[0])

(108, 240)
[3.139390e-10 2.721775e-10 1.430153e-11 3.709905e-01 -2.081011e-01
 3.237926e-12 4.145437e-10 -1.810879e-10 3.485628e-11 1.234879e-12
 -5.438619e-12 8.137850e-12 -5.568483e-11 -2.961208e-10 -8.361928e-10
 -7.800862e-12 -9.012840e-02 -5.727799e-12 6.894349e-10 -2.308952e-12
 6.157816e-02 -3.392241e-10 -2.966784e-11 -1.335898e-09 6.703053e-11
 3.695445e-10 2.261020e-10 3.478218e-12 -6.248709e-11 -4.533824e-03
 3.590077e-10 2.360759e-10 4.190389e-02 -6.336738e-12 -4.898829e-10
 8.528328e-11 1.711854e-10 -3.512660e-12 7.402165e-11 9.994408e-10
 -7.690594e-10 -1.212062e-10 -3.834504e-10 1.069500e-09 3.630058e-10
 -1.596711e-09 7.036907e-02 1.798452e-10 -3.289192e-12 4.874776e-11
 2.636545e-12 -2.002824e-10 -3.702246e-10 -1.855237e-10 -9.780424e-11
 -5.631498e-11 -4.928081e-10 -9.739734e-02 -3.222115e-10 -2.926004e-10
 1.760850e-10 5.425406e-11 -1.455395e-09 -2.727728e-10 -5.537713e-11
 1.290750e-10 1.282206e-10 -3.172647e-11 -3.609467e-12 -1.668066e-11
 4.422482e-02 -1.706346e-11

In [175]:
# descriptor grad
descriptor_grad = np.dot(l0_grad_tanh, w0.T)
print(descriptor_grad.shape)
print(descriptor_grad[0, :10])

(108, 2048)
[-5.517956e-01 -8.519849e-01 1.824896e-01 -5.835653e-01 -6.113289e-01
 2.390636e-01 1.038584e+00 -2.111849e-01 -1.862266e-01 -1.349426e+00]


***
## 3 反向 dscr 和 tabulate 模块
### 3.1 参数导入

In [176]:
def read_binary_tb_grad(filename):
    # 定义各个数组的大小
    s_g_sz = 108 * 512
    r_g_sz = 108 * 512 * 4
    fusa_g_sz = 108 * 128 * 4
    fusb_g_sz = 108 * 4 * 16
    descriptor_g_sz = 108 * 2048
    
    # 总大小
    total_sz = s_g_sz + r_g_sz + fusa_g_sz + fusb_g_sz + descriptor_g_sz
    
    # 从二进制文件中读取数据
    data = np.fromfile(filename, dtype=np.float64, count=total_sz)
    
    # 分割数据
    offset = 0
    
    s_g = data[offset:offset + s_g_sz].reshape((108, 512)).astype(np.float32)
    offset += s_g_sz
    
    r_g = data[offset:offset + r_g_sz].reshape((108, 512, 4)).astype(np.float32)
    offset += r_g_sz
    
    fusa_g = data[offset:offset + fusa_g_sz].reshape((108, 128, 4)).astype(np.float32)
    offset += fusa_g_sz
    
    fusb_g = data[offset:offset + fusb_g_sz].reshape((108, 4, 16)).astype(np.float32)
    offset += fusb_g_sz
    
    descriptor_g = data[offset:offset + descriptor_g_sz].reshape((108, 2048)).astype(np.float32)
    
    print("s_g shape:", s_g.shape)
    print("r_g shape:", r_g.shape)
    print("fusionA_g shape:", fusa_g.shape)
    print("fusionB_g shape:", fusb_g.shape)
    print("descriptor_g shape:", descriptor_g.shape)
    
    return s_g, r_g, fusa_g, fusb_g, descriptor_g

In [177]:
s_g, r_g, fusa_g, fusb_g, descriptor_g = read_binary_tb_grad("params/tb_grad.txt")

s_g shape: (108, 512)
r_g shape: (108, 512, 4)
fusionA_g shape: (108, 128, 4)
fusionB_g shape: (108, 4, 16)
descriptor_g shape: (108, 2048)


### 3.2 参数打印

In [178]:
print(f"A_g {fusa_g.shape}\n", fusa_g[0])
print(f"B_g {fusb_g.shape}\n", fusb_g[0])

A_g (108, 128, 4)
 [[-4.047016e-04 8.995893e-03 1.003011e-03 -5.481265e-04]
 [-3.945210e-03 -1.458901e-04 1.587017e-03 6.941293e-03]
 [-1.130329e-03 8.304412e-03 7.163507e-04 1.761712e-02]
 [-1.781580e-03 -7.374346e-03 -3.726070e-04 -6.428283e-04]
 [1.173580e-04 -8.856477e-03 -4.913725e-03 -1.232067e-03]
 [-1.088876e-04 -4.363899e-03 1.019602e-04 1.348623e-04]
 [-1.367737e-03 -2.146878e-03 -1.949265e-04 -1.698167e-03]
 [-5.722541e-03 -4.799653e-04 3.871797e-03 -1.270375e-04]
 [8.805793e-04 2.124952e-05 -7.047177e-05 6.166601e-04]
 [-1.064195e-03 -3.419461e-05 3.213287e-04 -4.549543e-04]
 [-4.781813e-04 -1.666285e-04 -7.452953e-04 7.649596e-05]
 [1.233104e-03 3.556166e-03 1.035218e-04 -6.045463e-04]
 [-3.480304e-04 4.317163e-03 2.695596e-03 -1.662249e-03]
 [-4.157652e-04 -5.604503e-03 -2.848391e-04 6.393321e-04]
 [5.276261e-04 4.475382e-03 4.912064e-05 6.673048e-04]
 [-5.899434e-03 5.460963e-04 7.846172e-04 -4.985404e-05]
 [3.152591e-04 1.052476e-03 1.857582e-06 3.440777e-04]
 [6.090529

In [179]:
print(descriptor_g)

[[-5.517957e-01 -8.519849e-01 1.824896e-01 ... -4.773968e-02 1.452616e+00
  1.699431e+00]
 [-5.688136e-01 -8.618886e-01 1.859186e-01 ... -4.806470e-02 1.468249e+00
  1.710196e+00]
 [-5.696402e-01 -8.578365e-01 1.837850e-01 ... -3.963074e-02 1.444935e+00
  1.687918e+00]
 ...
 [-5.638572e-01 -8.535649e-01 1.844922e-01 ... -4.957046e-02 1.411421e+00
  1.647296e+00]
 [-5.637113e-01 -8.562870e-01 1.827961e-01 ... -4.681514e-02 1.474478e+00
  1.720597e+00]
 [-5.849249e-01 -8.685721e-01 1.887551e-01 ... -4.674989e-02 1.455842e+00
  1.689247e+00]]


In [180]:
# 计算 fusa_grad fusb_grad 和软件代码执行结果对比
A_g = np.zeros((108, 4, 128))
B_g = np.zeros((108, 4, 16))
d_g = descriptor_g.reshape((108, 128, 16))

for i in range(108):
    A_g[i] = np.dot(fusb[i], d_g[i].T)
    B_g[i] = np.dot(fusa[i], d_g[i])

In [181]:
A_g[:, :, :16] += B_g[:, :, :16]

In [182]:
A_g *= 4/2048

In [183]:
print(f"A_g {A_g.shape}\n", A_g[0])
print(f"B_g {B_g.shape}\n", B_g[0])

A_g (108, 4, 128)
 [[-4.047018e-04 8.995893e-03 1.003011e-03 -5.481265e-04 -3.945210e-03
  -1.458903e-04 1.587017e-03 6.941293e-03 -1.130329e-03 8.304415e-03
  7.163504e-04 1.761712e-02 -1.781580e-03 -7.374345e-03 -3.726070e-04
  -6.428283e-04 1.173580e-04 -8.856477e-03 -4.913725e-03 -1.232067e-03
  -1.088876e-04 -4.363899e-03 1.019602e-04 1.348623e-04 -1.367737e-03
  -2.146878e-03 -1.949265e-04 -1.698167e-03 -5.722541e-03 -4.799654e-04
  3.871796e-03 -1.270376e-04 8.805793e-04 2.124954e-05 -7.047177e-05
  6.166601e-04 -1.064195e-03 -3.419462e-05 3.213286e-04 -4.549543e-04
  -4.781813e-04 -1.666285e-04 -7.452953e-04 7.649596e-05 1.233104e-03
  3.556166e-03 1.035218e-04 -6.045464e-04 -3.480304e-04 4.317163e-03
  2.695596e-03 -1.662248e-03 -4.157652e-04 -5.604503e-03 -2.848391e-04
  6.393321e-04 5.276261e-04 4.475383e-03 4.912063e-05 6.673049e-04
  -5.899435e-03 5.460963e-04 7.846172e-04 -4.985403e-05 3.152592e-04
  1.052476e-03 1.857567e-06 3.440777e-04 6.090528e-04 -2.051069e-03
  -1.1